In [ ]:
#@title Module version control (Run this cell only if code dosen't run properly)

# !pip install pandas==1.5.3
# !pip install matplotlib==3.7.1
# !pip install numpy==1.22.4
# !pip install ipywidgets==7.7.1
# !pip install google==2.0.3
# !pip install scipy==1.10.1

# Welcome to the EASI-ORC data analysis tool!
Please read the instructions carefully before each step.

Some things to keep in mind:
1.   Before you can start using the tool-set you must create a copy that will be saved under your google account (File > save a copy in drive).
2.   Always start from a clean slate. Use Runtime > Disconnect and delete runtime to remove all uploaded and saved files and Edit > Clear all outputs to clear all previous outputs.
3. This tool-set is ment to be used with data collected with the EASI-ORC image analysis scripts. Data from other sources will probably not analyze correctly or at all.
4. The different modules are ment to be activated linearily. Don't skip any.
5. On the top left, you'll find the folder icon (bottom of the icon column). Press it to get access to all tables and saved images. You can right-click and download any file from that menu, or download them all in a single zip file at the end.
6. The top module is a version control module. No need to activate it, unless you get a version control error in one of the other modules.
7. Pressing the angle bracket sign at the top left will display the module's code.


**In each module, a run icon is displayed at the top left. Press it to run the module. Wait until it's done (a green check mark will appear) before advancing.**


In [1]:
# from easiorc_functions import *
import easiorc_functions as easiorc
import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
# from ipywidgets import interactive, IntSlider, FloatSlider, widgets
import seaborn as sns
import scipy
import tkinter as tk
from tkinter import filedialog
import sys
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import os


# Upload Data

**Start** - Press the first run icon and upload your EASI-ORC result files. Each batch of files will be treated as a single strain or treatment. Make sure to select all result files of a single type (select all in the browse window, using the Shift key), with accordance to your experimental design.

**Name** - After each group of files is uploaded, you will be prompted to provide a name for it. This name will be used for it in the graphs and tables produced by this tool. Try to avoid overly long names.

**Order** - If you have a preference for the order in which the different treatments are presented in the graphs, make sure you upload your files by that order. The first will be the leftmost or top in all graphs and table and the last will be at the right or bottom.

**Output** - Once you're done, a joined table will be created for each group of files.

In [2]:
root = tk.Tk()
root.lift()
root.withdraw()
root.attributes('-topmost', True)
path = filedialog.askdirectory(parent = root, initialdir = '', title = "choose directory containing EASI-ORC results")
if path == '':
  sys.exit("No folder selected")
main_tables_list = glob.glob(path + "//*.csv")

while True:
  treatments_num = input("\nPlease type the number of treatments:\t")
  if treatments_num.isnumeric():
    treatments_num =int(treatments_num)
    print(f'\nExperiment consists of {treatments_num} treatment/s overall.\n')
    break
  else:
    print("\nPlease enter positive integers only.\n")

coverages_df_list, rna_df_list = [], []
treatment_idx = 0
while True:
  input_string=str(input("Please type treatment identidier string: "))
  tables_strain_lists=list(filter(lambda x: input_string in x, main_tables_list))
  if len(tables_strain_lists) == 0:
    print("No such treatment identifier found. Please try again.")
    continue
  treatment_name=str(input(f'Please type the name of treatment no.{treatment_idx+1}:\t'))
  treatment_name = treatment_name.replace("_", " ")
  temp_coverages_df = easiorc.prepare_dataframes(pd.concat([pd.read_csv(x) for x in tables_strain_lists])).reset_index(drop=True)
  temp_coverages_df['treatment'] = treatment_name
  temp_coverages_df['cell_id'] = temp_coverages_df.index.astype('int')
  coverages_df_list.append(temp_coverages_df)
  rna_df_temp = pd.concat([df for df in map(lambda idx: easiorc.rna_df_function(idx, df=temp_coverages_df), np.arange(len(temp_coverages_df))) if not df.isna().all(axis=None)])
  rna_df_temp['treatment'] = treatment_name
  rna_df_list.append(rna_df_temp)
  treatment_idx += 1
  if treatment_idx == treatments_num:
    break

coverages_df = pd.concat(coverages_df_list).drop(columns=['Cell #', 'Spots Coordinates Intensity and Colocalization (Far Near or Not Colocolized)']).reset_index(drop=True)
rna_df = pd.concat(rna_df_list).drop(columns=['status'], inplace=False).reset_index(drop=True).fillna(0, inplace=False)


Experiment consists of 3 treatment/s overall.



# Data Filtering
Before the data can be analayzed, it must be filtered.

**Important**
1. The filters in this section must be activated in order.
2. If you go back and update the filter parameters in a previous filter, you must run the next ones again for the new values to take effect.

**Instructions**
1. Click the run button of the filter you want to activate.
2. Change filter parameters (min and max values) by double clicking the value and writing your own, or with the slider, according to your experimental design and each filter's specific notes. The changes will be updated in the plot as you make them.
3. Notice you can change the names of many parameter in each figure. If you change any names, make sure to re-run the cell for the changes to take effect.
You can also change many design parameters, including font style and sizes in different sections of the plot.
4. To save a figure, click the save button below each figure. It will be saved with the filter parameters in the file's name in the 'Figures' folder.
5. Once you're happy with the parameters you've chosen and saved an image of the plot, you can continue to the next filter.

# Filter 1 - smFISH signal intesities
To make sure the spots identifed using EASI-ORC are all true signals, you must filter out any outliers identified, using their intesitiy. Both high and low intesitiy spots identified my be noise.

Aim to keep most signals, while removing outliers.

There are no predetermined values to reccomend, since they change by the protocol used, flourophore chosen and imaging equipment.

In [3]:
# @title  { display-mode: "form" }

fig_name =  'smFISH Signal Intensity Organelle Comparison ' #@param {type:"string"}
main_title_label = 'mRNA signal Intensity' #@param {type:"string"}
x_axis_label = 'mRNA signal intensities' #@param {type:"string"}
y_axis_label = 'Signals count' #@param {type:"string"}


def plot_function(main_title=22, axis_labels=16, axis_ticks = 16, legend=12, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind', _min_=0, _max_=max(rna_df['intensity'])+1, bins=100):   
    
    plt.figure(figsize=(10, 3))
    plt.style.use(plot_style)
    for sample in list(rna_df['treatment'].unique()):
        sns.histplot(data=rna_df[rna_df['treatment']==sample], 
                     x='intensity', 
                     bins=bins,
                     edgecolor=None, 
                     alpha=.3, 
                     label=f'{sample}, signals: {len(rna_df[rna_df["intensity"].between(_min_, _max_) & (rna_df["treatment"]==sample)])}',
                     )   
    
    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)
    plt.title(f'Total signals presented : {len(rna_df[rna_df['intensity'].between(_min_, _max_)])}\nTotal signals omitted:  {len(rna_df)-len(rna_df[rna_df['intensity'].between(_min_, _max_)])}', y= -0.49)
    plt.xlim([_min_-.5, _max_+.5])
    easiorc.global_assist(min=_min_, max=_max_, DPI=DPI, fig_name=fig_name, file_type=file_type, path=path)

    global intensity_threshold
    intensity_threshold = [_min_,_max_]
    
    plt.show() 

easiorc.show_interactive(plot_function, min=(0,max(rna_df['intensity'])-.1), max=(0,max(rna_df['intensity'])+1), slider_value=(0, max(rna_df['intensity'])+1), step=.01, bins=easiorc.bins)


# Filter 2 - Organelle covrage per cell, per z-plane
Depending on the experimental design, the observed organelle will likely appear different between cells. Some may have very little or no organelle signal, while other may be completly covered by it. For each cell, you'd want to disregard the planes where these outliers appear.

Analyzing a plane where the entire cell is covered with an organelle will yield a false positive identification of colocalization between the mRNA signals and the organelle. Similiarly, using a plane where no organelle signal is observed will create the opposite bias, resolving all mRNA signals and not colocolized.

The precise values you should choose are determined by your specific exeprimental makeup (organelle observed, imaging tools used, etc).

In [4]:
rna_df_filtered_intensities = rna_df[rna_df['intensity'].between(*intensity_threshold)].copy()


fig_name =  'Organelle coverage by cross-sections fig' #@param {type:"string"}
main_title_label = 'Organelle coverage of cell per cross-section' #@param {type:"string"}
x_axis_label = 'Organelle coverage (%)' #@param {type:"string"}
y_axis_label = 'Cross-sections count' #@param {type:"string"}


def plot_function(main_title=22, axis_labels=16, axis_ticks=16, legend=12, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind', _min_=0, _max_=100, bins=100):
    plt.figure(figsize=(10, 3))
    plt.style.use(plot_style)
    for sample in rna_df['treatment'].unique():
        sample_array= np.array(coverages_df[coverages_df['treatment']==sample].iloc[:,:-2]).flatten()
        sample_array = sample_array[(sample_array >= _min_) & (sample_array <= _max_)]
        sns.histplot(x=sample_array, 
                     bins=bins,
                     edgecolor=None, 
                     alpha=.3, 
                     label=f'{sample}, cross-sections: {len(sample_array[(sample_array >= _min_) & (sample_array <= _max_)])}',)
    
    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)
    
    coverage_array = np.array(coverages_df.iloc[:,:-2]).flatten()
    plt.xlim([_min_-.5, _max_+.5])

    plt.title(f'Total sections presented : {len(coverage_array[(coverage_array >= _min_) & (coverage_array <= _max_)])}\nTotal sections omitted:  {len(coverage_array)-len(coverage_array[(coverage_array >= _min_) & (coverage_array <= _max_)])}', y= -0.49)
    easiorc.global_assist(min=_min_, max=_max_, DPI=DPI, fig_name=fig_name, file_type=file_type, path=path)
    
    global coverage_threshold
    coverage_threshold = [_min_,_max_]  
    
    plt.show() 

easiorc.show_interactive(plot_function, min=(0,100), max=(0,100), slider_value=(.01, 100), step=.01, bins=easiorc.bins)

# Filter 3 - smFISH Signals Per Cell
The final filter allows you to only count cells with a specific number of smFISH signals in them.

The values chosen for this filter will depend upon your target mRNA and experimental procedures. For example, if you target mRNA is very abundent under your experimental growth conditions, you'd want to include cells with severl dozens of signals.

As usual, you may want to remove any outliers. If a cell has hunderds of singals, it may be the result of noise, and you'll want to remove it.

In [5]:
filtered_rna_df = rna_df[(rna_df['intensity'].between(*intensity_threshold)) & (rna_df['organelle_coverage'].between(*coverage_threshold))].copy()
cells_df = easiorc.rna_to_cell_df(filtered_rna_df)

fig_name_signals =  'mRNA signals per cell fig' #@param {type:"string"}
main_title_label = 'mRNA signals per cell' #@param {type:"string"}
x_axis_label = 'mRNA signals per cells' #@param {type:"string"}
y_axis_label = 'Cells count' #@param {type:"string"}

def plot_function(main_title=22, axis_labels=16, axis_ticks=16, legend=12, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind', _min_=0, _max_=max(cells_df['rna_count'])):

    plt.figure(figsize=(10, 3))
    plt.style.use(plot_style)
    for sample in list(cells_df['treatment'].unique()):
        sample_array = np.array(cells_df[(cells_df["rna_count"].between(_min_, _max_)) & (cells_df["treatment"] == sample)]['rna_count']).flatten()
        sns.histplot(x=sample_array,
                     edgecolor=None, 
                     alpha=.3, 
                     label=f'{sample}, cells: {len(cells_df[(cells_df["rna_count"].between(_min_, _max_)) & (cells_df["treatment"] == sample)])}')   
        
    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)
    plt.xlim([_min_-.5, _max_+.5])
    
    plt.title(f'Total cells presented : {len(cells_df[cells_df['rna_count'].between(_min_, _max_)])}\nTotal cells omitted:  {len(cells_df)-len(cells_df[cells_df['rna_count'].between(_min_, _max_)])}', y= -0.49)
    easiorc.global_assist(min=_min_, max=_max_, DPI=DPI, fig_name=fig_name_signals, file_type=file_type, path=path)

    global rna_num_threshold
    rna_num_threshold = [_min_,_max_]
    
    plt.show() 

easiorc.show_interactive(plot_function, type='int', min=(0, max(cells_df['rna_count'])), max=(0, max(cells_df['rna_count'])), slider_value=(1,max(cells_df['rna_count'])), step=1)

# Filtered Data Production



*  After you're happy with your filtering parameters, run this cell to produce filtered data.
*  If you change filters, you must run this cell again, to update the data accordingly.

# Figures
In the next few cells, you will be able to use the filtered data to present several figures.

Each can be saved using the 'Save Figure' button below the figure.

Notice you can change the names of many parameter in each figure. If you change any names, make sure to re-run the cell for the changes to take effect.
You can also change many design parameters, including font style and sizes in different sections of the plot.

Lastly, before saving, you can change the file's type, quality and even choose a different plot style.

Actual value will be availabe in the stats table created in the end of the tool-set (including error values).

# mRNA-Organelle Colocalization (Stacked Bar Plot)
After filtering out any unwanted signals, cells and z-planes, your data can be presented graphically. The first figure is a stacked bar plot that shows the colocalization precentages of each starin or treatment (group of files uploaded).

Standard error is displayed for each of the stacked bars in a corresponding, darker color.

In [20]:
# @title  { display-mode: "form" }

# Plot: Colocalization proportions

filtered_cells_df = cells_df[cells_df['rna_count'].between(*rna_num_threshold)].copy()
grouped_df = filtered_cells_df[['treatment', 'not_colocalized', 'organelle_near_colocalized', 'organelle_far_colocalized']]
grouped_df = grouped_df.groupby('treatment').mean()
grouped_df = grouped_df.reset_index()

grouped_df['treatment'] = pd.Categorical(grouped_df['treatment'], categories=list(filtered_cells_df['treatment'].unique()), ordered=True)
grouped_df = grouped_df.sort_values('treatment')

fig_name = 'mRNA to organells colocalization proportion ' #@param {type:"string"}
main_title_label = 'mRNA-Organelle Colocalization (Stacked Bar Plot)' #@param {type:"string"}
x_axis_label = ''  #@param {type:"string"}
y_axis_label = 'Localization proportion' #@param {type:"string"}
legend_non_localized = 'Not ER Localized' #@param {type:"string"}
legend_close_localized = 'nER' #@param {type:"string"}
legend_far_localized = 'cER' #@param {type:"string"}

def plot_function(main_title=22, axis_labels=18, axis_ticks=16, legend=16, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):

    plt.style.use(plot_style)

    grouped_df.plot(x='treatment', kind='bar', stacked=True, figsize=(len(list(cells_df['treatment'].unique()))*1,4), legend=True, width=.95)
    error_bar_color='k'
    capsize=5
    
    for treatment in filtered_cells_df['treatment'].unique():
            plt.errorbar(x=treatment, y=grouped_df[grouped_df['treatment']==treatment]['not_colocalized'], 
                        yerr=scipy.stats.sem(filtered_cells_df[filtered_cells_df['treatment']==treatment]['not_colocalized']), color=error_bar_color, capsize=capsize)
            plt.errorbar(x=treatment, y=grouped_df[grouped_df['treatment']==treatment]['not_colocalized']+grouped_df[grouped_df['treatment']==treatment]['organelle_near_colocalized'], 
                        yerr=scipy.stats.sem(filtered_cells_df[filtered_cells_df['treatment']==treatment]['organelle_near_colocalized']), color=error_bar_color, capsize=capsize)
            if sum(filtered_cells_df['organelle_far_colocalized']) > 0:
                plt.errorbar(x=treatment, y=grouped_df[grouped_df['treatment']==treatment]['not_colocalized']+grouped_df[grouped_df['treatment']==treatment]['organelle_near_colocalized']+grouped_df[grouped_df['treatment']==treatment]['organelle_far_colocalized'], 
                            yerr=scipy.stats.sem(filtered_cells_df[filtered_cells_df['treatment']==treatment]['organelle_far_colocalized']), color=error_bar_color, capsize=capsize)

    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)    
    plt.xticks(fontsize=axis_ticks, rotation=30, ha="right")

    if sum(filtered_cells_df['organelle_far_colocalized']) > 0:
        plt.legend([legend_non_localized, legend_close_localized, legend_far_localized], fontsize=legend, loc='center left', bbox_to_anchor=(1, 0.5))
    else:
        plt.legend([legend_non_localized, legend_close_localized], fontsize=legend, loc='center left', bbox_to_anchor=(1, 0.5))

    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)
    
    plt.show()

easiorc.show_interactive(plot_function)

# mRNA-organelle Colocalization (violing plots)

This plot presents the same data points (colocalization proportions), in the form of violing plots. Each colocaliation designation is presented in a seperate sub-plot.

Each spot indicates a single cell in the filtered data.
The dashed white lines are the 25, 50, and 75 quartalised and the solid black line is the mean.

* Make sure you scroll down for the 'Save Figure' button.

In [8]:
# @title  { display-mode: "form" }

fig_name = 'mRNA to organells colocalization proportion violin fig' #@param {type:"string"}
main_title_label = '' #@param {type:"string"}
coloc_title_label = 'Organelle Localization' #@param {type:"string"}
non_coloc_title_label = 'Non-Organelle Localization' #@param {type:"string"}
close_coloc_title_label = 'Proximal Organelle Localization' #@param {type:"string"}
far_coloc_title_label = 'Distal Organelle Localization' #@param {type:"string"}
x_axis_label = ''  #@param {type:"string"}
y_axis_label = 'Proportion' #@param {type:"string"}

violin_columns = [col for col in filtered_cells_df.columns if 'local' in col]
violin_titles = [coloc_title_label, non_coloc_title_label, close_coloc_title_label, far_coloc_title_label]

if sum(filtered_cells_df['organelle_far_colocalized']) == 0:
    print('yeeees')
    violin_columns=violin_columns[:2]
    violin_titles=violin_titles[:2]

def plot_function(main_title=22, axis_labels=18, axis_ticks=16, legend=16, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):

    plt.style.use(plot_style)
    plt.figure(figsize=((len(list(filtered_cells_df['treatment'].unique()))*1.75)+2, len(violin_columns)*4))
    for idx, column in enumerate(violin_columns):
        plt.subplot(len(violin_columns), 1, idx+1).set_title(violin_titles[idx], fontsize=main_title)

        means = filtered_cells_df.groupby('treatment')[column].mean().values
        medians = filtered_cells_df.groupby('treatment')[column].median().values
        categories = sorted(filtered_cells_df['treatment'].unique())

        sns.violinplot(data=filtered_cells_df, x='treatment', y=column, hue='treatment', inner=None, edgecolor=None)
        sns.stripplot(data=filtered_cells_df, x='treatment', y=column, hue='treatment', jitter=0.35, alpha=.25, edgecolor='k', linewidth=1, size=3)
        plt.scatter(categories, means, color='k', edgecolor='w', linewidth=.9, label='Mean', zorder=3, marker='o', s=90)
        plt.scatter(categories, medians, color='k', edgecolor='w', linewidth=.9, label='Median', zorder=3, marker='X', s=90)
        plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

        easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label) 
        plt.xticks(fontsize=axis_ticks, rotation=30, ha="right")    
    
    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)

    plt.tight_layout()
    plt.show()

easiorc.show_interactive(plot_function)

# mRNA Signals per Cell
This violing plot presents the number of smFISH signals identifed per cell. Each spot represents a single cell.

In [9]:
# @title  { display-mode: "form" }

fig_name = 'mRNA signals per cell violin fig' #@param {type:"string"}
main_title_label = 'mRNA Signals per Cell' #@param {type:"string"}
x_axis_label = 'Strains' #@param {type:"string"}
y_axis_label = 'mRNA signals per cell' #@param {type:"string"}

categories = sorted(filtered_cells_df['treatment'].unique())
means = filtered_cells_df.groupby('treatment')['rna_count'].mean().values
medians = filtered_cells_df.groupby('treatment')['rna_count'].median().values

def plot_function(main_title=22, axis_labels=18, axis_ticks=16, legend=16, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):
    plt.style.use(plot_style)
    plt.figure(figsize=(len(list(filtered_cells_df['treatment'].unique())), 5))
    sns.violinplot(data=filtered_cells_df, x='treatment', y='rna_count', hue='treatment', inner=None, edgecolor=None)
    sns.stripplot(data=filtered_cells_df, x='treatment', y='rna_count', hue='treatment', jitter=0.35, alpha=.25, edgecolor='k', linewidth=1, size=3)
    plt.scatter(categories, means, color='k', edgecolor='w', linewidth=.9, label='Mean', zorder=3, marker='o', s=90)
    plt.scatter(categories, medians, color='k', edgecolor='w', linewidth=.9, label='Median', zorder=3, marker='X', s=90)
    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)
    plt.xticks(fontsize=axis_ticks, rotation=30, ha="right")
    plt.legend(fontsize=legend, loc='center left', bbox_to_anchor=(1, 0.5))

    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)

easiorc.show_interactive(plot_function)

# Average Organelle Coverage per Cell
This violing plot presents the avarage coverage (in %) of an organelle per cell. Only the z-planes used in the analysis (those remianing after filtering) are used for the calculation.

In [10]:
# @title  { display-mode: "form" }

fig_name = 'Average Organelle Coverage per Cell violin' #@param {type:"string"}
main_title_label = 'Average Organelle Coverage per Cell' #@param {type:"string"}
x_axis_label = 'Strains' #@param {type:"string"}
y_axis_label = 'Average Organelle Coverage (%)' #@param {type:"string"}

filtered_cells_df['cell_id'] = filtered_cells_df['cell_id'].astype(int)
final_cells_df = filtered_cells_df.merge(coverages_df, on=['treatment', 'cell_id'])
final_cells_df_columns = [col for col in final_cells_df.columns if 'Organelle Signal Size Cross Section' in col]
final_cells_df = final_cells_df.apply(lambda x: easiorc.filtered_mean_coverages(row=x, coverage_threshold=coverage_threshold, cols=final_cells_df_columns), axis=1)


def plot_function(main_title=22, axis_labels=18, axis_ticks=16, legend=16, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):
    plt.style.use(plot_style)
    plt.figure(figsize=(len(list(filtered_cells_df['treatment'].unique())), 5))

    categories = final_cells_df['treatment'].unique()
    means = [final_cells_df[final_cells_df['treatment'] == category]['filtered_mean_organelle_coverage'].mean() for category in categories]
    medians = [final_cells_df[final_cells_df['treatment'] == category]['filtered_mean_organelle_coverage'].median() for category in categories]

    sns.violinplot(data=final_cells_df, x='treatment', y='filtered_mean_organelle_coverage', hue='treatment', inner=None, edgecolor=None)
    sns.stripplot(data=final_cells_df, x='treatment', y='filtered_mean_organelle_coverage', hue='treatment', jitter=0.35, alpha=.25, edgecolor='k', linewidth=1, size=3)
    plt.scatter(categories, means, color='k', edgecolor='w', linewidth=.9, label='Mean', zorder=3, marker='o', s=90)
    plt.scatter(categories, medians, color='k', edgecolor='w', linewidth=.9, label='Median', zorder=3, marker='X', s=90)

    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)
    plt.xticks(fontsize=axis_ticks, rotation=30, ha="right")
    plt.legend(fontsize=legend, loc='center left', bbox_to_anchor=(1, 0.5))

    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)

    plt.show()

easiorc.show_interactive(plot_function)


# Correlations Between Major Data Points
The next two plots provide an examination of two useful correlations:

The first is between the relative organelle coverage (avarage % of cell surface within filtered parameters) and the mRNA-organelle colocalization proportion in that cell.

The second is between the number of smFISH signals in a cell and the mRNA-organelle colocalization proportion in that cells.

Each point in these graphs corresponts to a single cell.

These may help you identify the existance or lack-there-off of technical biases.

# Organelle Coverage vs mRNA-organelle Colocalization

In [11]:
# @title  { display-mode: "form" }

fig_name = 'mean organelle coverage vs. organelle localization' #@param {type:"string"}
main_title_label = 'Mean Organelle Coverage vs. Organelle Localization' #@param {type:"string"}
x_axis_label = 'Mean organelle coverage (%)' #@param {type:"string"}
y_axis_label = 'mRNA-organelle localization (%)' #@param {type:"string"}

def plot_function(main_title=22, axis_labels=16, axis_ticks=16, legend=12, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):
    plt.style.use(plot_style)
    plt.figure()

    for treatment in final_cells_df['treatment'].unique():
        curr_df = final_cells_df[final_cells_df['treatment']==treatment]
        curr_df = final_cells_df[final_cells_df['treatment']==treatment]
        x= curr_df['filtered_mean_organelle_coverage'] + 0.8 * np.random.rand(len(curr_df['filtered_mean_organelle_coverage'])) 
        y= curr_df['colocalized']*100 + 0.8 * np.random.rand(len(curr_df['colocalized']))

        sns.scatterplot(data=curr_df, x=x, y=y, 
                        linewidth= 0.2, s=20, alpha=.5, label=f'{treatment}, R = {round(scipy.stats.pearsonr(curr_df['filtered_mean_organelle_coverage'],curr_df['colocalized'])[0], 3)}')
        easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)

    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)
    
    plt.show()

easiorc.show_interactive(plot_function)

# mRNA Signals per Cell vs mRNA-organelle Colocalization

In [12]:
# @title  { display-mode: "form" }

fig_name = 'mRNA signals vs. organelle localization' #@param {type:"string"}
main_title_label = 'mRNA signals per cell vs. organelle localization' #@param {type:"string"}
x_axis_label = 'mRNA signals per cell' #@param {type:"string"}
y_axis_label = 'mRNA-organelle localization (%)'#@param {type:"string"}

def plot_function(main_title=22, axis_labels=16, axis_ticks=16, legend=12, file_type='png', DPI=1000, plot_style='seaborn-v0_8-colorblind'):
    plt.style.use(plot_style)
    plt.figure()
    for treatment in final_cells_df['treatment'].unique():
        curr_df = final_cells_df[final_cells_df['treatment']==treatment]
        x= curr_df['rna_count'] + .8 * np.random.rand(len(curr_df['rna_count']))
        y= curr_df['colocalized']*100 + 0.8 * np.random.rand(len(curr_df['colocalized']))
        sns.scatterplot(data=curr_df, x=x, y=y, 
                        linewidth= 0.2, s=20, alpha=.5, 
                        label=f'{treatment}, R = {round(scipy.stats.pearsonr(curr_df['rna_count'],curr_df['colocalized'])[0], 3)}'
                        )
    easiorc.plot_assist(axis_ticks, axis_labels, legend, main_title, x_axis_label, y_axis_label, main_title_label)

    easiorc.global_assist( DPI=DPI, fig_name=fig_name, file_type=file_type, filter=False, intensity_threshold=intensity_threshold, coverage_threshold=coverage_threshold, rna_num_threshold=rna_num_threshold, path=path)



# interactive_plot_mRNA_signals_vs_localization = interactive(plot_mRNA_signals_vs_localization, file_type=easiorc.file_type, axis_ticks=easiorc.axis_ticks, DPI=easiorc.DPI, main_title=easiorc.main_title, axis_labels=easiorc.axis_labels, legend=easiorc.legend, plot_style=easiorc.graph_styles)
# output = interactive_plot_mRNA_signals_vs_localization.children[-1]
# widgets.VBox([interactive_plot_mRNA_signals_vs_localization, easiorc.save_btn])

easiorc.show_interactive(plot_function)

# Statistacal Summery
This cell creates a summery table which includes all values used in the figures created above. This includes the accurate proportion and error values calulcated for them.

In addition, it presents t-tests between each of the different strains/treatments in you experiment, as-well-as the p-values of the correlation plots.

In [13]:
# @title  { display-mode: "form" }

final_cells_df_copy = final_cells_df.copy()
rna_df_copy = rna_df.copy()
filtered_rna_df_copy = filtered_rna_df.copy()

if sum(final_cells_df_copy['organelle_far_colocalized']) == 0:
    final_cells_df_copy.drop(columns = ['organelle_far_colocalized', 'organelle_near_colocalized'], inplace = True)
    rna_df_copy.drop(columns = ['organelle_far_colocalized', 'organelle_near_colocalized'], inplace = True)
    filtered_rna_df_copy.drop(columns = ['organelle_far_colocalized', 'organelle_near_colocalized'], inplace = True)

stats_df = pd.DataFrame()
for idx, treatment in enumerate(filtered_cells_df['treatment'].unique()):
    stats_df.loc[idx, 'Feature'] = treatment
    stats_df.loc[idx, 'Number of cells before filtering'] = len(coverages_df[coverages_df['treatment']==treatment])
    stats_df.loc[idx, 'Number of cells after filtering'] = len(filtered_cells_df[filtered_cells_df['treatment']==treatment])
    stats_df.loc[idx, 'Percentage of positive cells'] = stats_df.loc[idx, 'Number of cells after filtering']/stats_df.loc[idx, 'Number of cells before filtering']*100
    stats_df.loc[idx, 'mean proportion of organelle localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'].mean()
    stats_df.loc[idx, 'standard deviation proportion of organelle localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'].std()
    stats_df.loc[idx, 'standard error proportion of organelle localization'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'])
    stats_df.loc[idx, 'mean proportion of non-organelle localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['not_colocalized'].mean()
    stats_df.loc[idx, 'standard deviation proportion of non-organelle localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['not_colocalized'].std()
    stats_df.loc[idx, 'standard error proportion of non-organelle localization'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['not_colocalized'])

    if sum(final_cells_df['organelle_far_colocalized'])>0:
        stats_df.loc[idx, 'mean proportion of organelle near localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_near_colocalized'].mean()
        stats_df.loc[idx, 'standard deviation proportion of organelle near localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_near_colocalized'].std()
        stats_df.loc[idx, 'standard error proportion of organelle near localization'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_near_colocalized'])
        stats_df.loc[idx, 'mean proportion of organelle far localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_far_colocalized'].mean()
        stats_df.loc[idx, 'standard deviation proportion of organelle far localization'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_far_colocalized'].std()
        stats_df.loc[idx, 'standard error proportion of organelle far localization'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['organelle_far_colocalized'])

    stats_df.loc[idx, 'mean mRNA signals per cell'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['rna_count'].mean()
    stats_df.loc[idx, 'standard deviation mRNA signals per cell'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['rna_count'].std()
    stats_df.loc[idx, 'standard error mRNA signals per cell'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['rna_count'])
    stats_df.loc[idx, 'mean organelle coverage'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['filtered_mean_organelle_coverage'].mean()
    stats_df.loc[idx, 'standard deviation organelle coverage'] = final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['filtered_mean_organelle_coverage'].std()
    stats_df.loc[idx, 'standard error organelle coverage'] = scipy.stats.sem(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['filtered_mean_organelle_coverage'])
    stats_df.loc[idx, 'Pearson correlation mean organelle coverages vs organelle localizations'] = scipy.stats.pearsonr(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['filtered_mean_organelle_coverage'], final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'])[0]
    stats_df.loc[idx, 'Correlation pvalue mean organelle coverages vs organelle localizations'] =  scipy.stats.pearsonr(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['filtered_mean_organelle_coverage'], final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'])[1]
    stats_df.loc[idx, 'Pearson correlation mean mRNA signals vs organelle localizations'] = scipy.stats.pearsonr(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['rna_count'], final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'])[0]
    stats_df.loc[idx, 'Correlation pvalue mean mRNA signals vs organelle localizations'] =  scipy.stats.pearsonr(final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['rna_count'], final_cells_df_copy[final_cells_df_copy['treatment']==treatment]['colocalized'])[1]

os.makedirs(os.path.join(path, 'Tables'), exist_ok=True)
if len(final_cells_df_copy['treatment'].unique()) > 1:
    stats_columns = [col for col in final_cells_df_copy.columns if 'local' in col] + list(['rna_count', 'filtered_mean_organelle_coverage'])
    tukey_list = []
    for column in stats_columns:
        comparison_list_list = list(final_cells_df_copy[final_cells_df_copy['treatment']==treatment][column] for treatment in final_cells_df_copy['treatment'].unique())
        statistic, pvalue = scipy.stats.f_oneway(*comparison_list_list)
        stats_df[f'ANOVA (one-way) {column} pvalue'] = pvalue
        stats_df[f'ANOVA (one-way) {column} statistic'] = statistic
        tukey_result = pairwise_tukeyhsd(endog=final_cells_df_copy[column], groups=final_cells_df_copy['treatment'], alpha=0.05)
        tukey_temp = pd.DataFrame(data=tukey_result.summary().data[1:], columns=tukey_result.summary().data[0])
        tukey_temp['comparison'] = column
        tukey_list.append(tukey_temp)
    tukey_df=pd.concat(tukey_list).reset_index(drop=True)
    tukey_col =  tukey_df['comparison'].copy()
    tukey_df.drop(columns='comparison', inplace=True)
    tukey_df.insert(0, 'comparison', tukey_col)
    
    stats_df.to_csv(os.path.join(path, 'Tables', 'statistics_summary.csv'), index=False, header=True)
    tukey_df.to_csv(os.path.join(path, 'Tables', 'tukey_summary.csv'), index=False, header=True)

final_cells_df_copy.to_csv(os.path.join(path, 'Tables', 'filtered_cells_df.csv'), index=False, header=True)
rna_df_copy.to_csv(os.path.join(path, 'Tables', 'unfiltered_rna_df.csv'), header=True)
filtered_rna_df_copy.to_csv(os.path.join(path, 'Tables', 'filtered_rna_df.csv'), header=True)

Thank you for using the EASI-ORC data analysis tool. For any questions or comments, please contact the author of this tool, Liav Levavi, at liav.levavi@weizmann.ac.il.